In [ ]:
# https://medium.com/@geonextgis/getting-started-with-microsoft-planetary-computer-stac-api-67cbebe96e5e

In [ ]:
import pystac_client
import planetary_computer
import odc.stac
import matplotlib.pyplot as plt

from pystac.extensions.eo import EOExtension as eo

In [ ]:
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']
plt.rcParams["font.size"] = 14

In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace
)

In [ ]:
bbox_of_interest = [77.3521, 12.7235, 77.8477, 13.2186]
time_of_interest = "2022-01-01/2022-12-31"

In [ ]:
# Query the catalog and sort the filtered image collection based on 'cloud_cover' 
search = catalog.search(
    collections=["landsat-c2-l2"],
    bbox=bbox_of_interest,
    datetime=time_of_interest,
    query={"eo:cloud_cover": {"lt": 10}},
    sortby=["eo:cloud_cover"]
)

items = search.item_collection()
print(f"Returned {len(items)} Items")

In [ ]:
# Take the first item with minimum cloud cover
selected_item = items[0]

print(
    f"Choosing {selected_item.id} from {selected_item.datetime.date()}"
    + f"with {selected_item.properties['eo:cloud_cover']}% cloud cover"
)

In [ ]:
for key, asset in selected_item.assets.items():
    print(f"{key}: {asset.title}")

In [ ]:
bands_of_interest = ["nir08", "red", "green", "blue", "qa_pixel", "lwir11"]

data = odc.stac.stac_load(
    [selected_item],
    bands=bands_of_interest,
    bbox=bbox_of_interest
).isel(time=0)

In [ ]:
data

In [ ]:
rgb_data_array = data[["red", "green", "blue"]].to_array()

fig, ax = plt.subplots(figsize=(10, 10))
rgb_data_array.plot.imshow(robust=True, ax=ax)
ax.set_title("Natural Color, Bengaluru, Karnataka, India")

In [ ]:
sfcc_data_array = data[["nir08", "red", "green"]].to_array()

fig, ax = plt.subplots(figsize=(10, 10))
sfcc_data_array.plot.imshow(robust=True, ax=ax)
ax.set_title("Standard False Color, Bengaluru, Karnataka, India")

## Render NDVI and NDWI images of the AOI

In [ ]:
green = data["green"].astype("float")
red = data["red"].astype("float")
nir = data["nir08"].astype("float")

ndvi = (nir - red) / (nir + red)
ndwi = (green - nir) / (green + nir)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(28, 12))
axes = axes.flatten()

ndvi.plot.imshow(ax=axes[0], robust=True, cmap="RdYlGn", add_colorbar=True, cbar_kwargs={"aspect": 30, "pad": 0.03, "label": "NDVI"})
axes[0].set_title("NDVI, Bengaluru, Karnataka, India")

ndwi.plot.imshow(ax=axes[1], robust=True, cmap="YlGnBu", add_colorbar=True, cbar_kwargs={"aspect": 30, "pad": 0.03, "label": "NDWI"})
axes[1].set_title("NDWI, Bengaluru, Karnataka, India")

plt.tight_layout()

## Selecting specific platforms

In [ ]:
catalog.get_collection("landsat-c2-l2").summaries.to_dict()["platform"]

In [ ]:
search = catalog.search(
    collections=["landsat-c2-l2"],
    bbox=bbox_of_interest,
    datetime=time_of_interest,
    query={
        "eo:cloud_cover": {"lt": 10},
        "platform": {"in": ['landsat-8', 'landsat-9']}
    },
    sortby=["eo:cloud_cover"]
)

items = search.item_collection()

## Rescaling Temperature Data

In [ ]:
band_info = selected_item.assets["lwir11"].extra_fields["raster:bands"][0]
band_info

In [ ]:
temperature = data["lwir11"].astype("float")
temperature *= band_info["scale"]
temperature += band_info["offset"]
temperature

In [ ]:
celcius = temperature - 273.15

plt.figure(figsize=(14, 12))
celcius.plot.imshow(cmap="magma", robust=True, cbar_kwargs={"aspect": 30, "pad": 0.03, "label": "Temperature (°C)"})
plt.title("Thermal, Bengaluru, Karnataka, India")